In [24]:
from sklearn.ensemble import RandomForestClassifier #para implementar el random forest
import numpy as np
import pandas
from sklearn import cross_validation

data = pandas.read_csv('./NO_C_2017.csv')
data1 = pandas.read_csv('./A.csv')
data2 = pandas.read_csv('./B.csv')
data3 = pandas.read_csv('./C.csv')
data4 = pandas.read_csv('./D.csv')

#unir 
datos = pandas.concat([data, data1, data2, data3, data4])

#mesclar los datos
datos = datos.sample(frac = 1)
datos = datos.sample(frac = 1)
datos = datos.sample(frac = 1)

datos = datos.reset_index(drop=True)

selectOpt = ["status_salud_publica","est_civil","sexo_desc","edad","estrato",
"ind_interd","comuna","ind_region_rm","avaluo_bbrr","cant_bbrr","avaluo_auto","cant_autos",
"n_actividad","n_rubros","clean2","tot_docs","tot_mont","ind_morosidad1","ind_morosidad2","ind_consultas_id",
"cant_personas_fam","cant_hijos_fam"]

datos.loc[((datos["est_civil"] == " ") & (datos["cant_hijos_fam"] == 0)), "est_civil"] = "SOLTERO"
datos.loc[((datos["est_civil"] == " ") & (datos["cant_hijos_fam"] > 0)), "est_civil"] = "CASADO"

#son pasados los estados a una variable binaria
datos.loc[datos["status_salud_publica"] == "S", "status_salud_publica"]= 1
datos.loc[datos["status_salud_publica"] == "N", "status_salud_publica"]= 0

#cambair sexo de la persona
datos.loc[datos["sexo_desc"] == "F", "sexo_desc"]= 0
datos.loc[datos["sexo_desc"] == "M", "sexo_desc"]= 1
datos.loc[datos["sexo_desc"] == "SI" , "sexo_desc"] = 0

#ind_interd
datos.loc[datos["ind_interd"] == " " , "ind_interd"] = 0
datos.loc[datos["ind_interd"] == "N" , "ind_interd"] = 0
datos.loc[datos["ind_interd"] == "S" , "ind_interd"] = 1

#gente de fuera de santiago agregar comuna
datos.loc[datos["est_civil"] == "SOLTERO", "est_civil"]= 0
datos.loc[datos["est_civil"] == "CASADO", "est_civil"]= 3
datos.loc[datos["est_civil"] == "VIUDO", "est_civil"]= 1
datos.loc[datos["est_civil"] == "DIVORCIADO", "est_civil"]= 2
datos.loc[datos["est_civil"] == "SEPARADO JUDICIALMENTE", "est_civil"]= 2

#son discretizados los estratos sociales
datos.loc[datos["estrato"] == "SIN CLASIFICACION", "estrato"]= 3
datos.loc[datos["estrato"] == "ABC1", "estrato"]= 5
datos.loc[datos["estrato"] == "C2", "estrato"]= 4
datos.loc[datos["estrato"] == "C3", "estrato"]= 3
datos.loc[datos["estrato"] == "D", "estrato"]= 2
datos.loc[datos["estrato"] == "E", "estrato"]= 1

#se indica que son columnas enteras
for name in selectOpt:
    print(name)
    datos[name] = datos[name].astype('int')

status_salud_publica
est_civil
sexo_desc
edad
estrato
ind_interd
comuna
ind_region_rm
avaluo_bbrr
cant_bbrr
avaluo_auto
cant_autos
n_actividad
n_rubros
clean2
tot_docs
tot_mont
ind_morosidad1
ind_morosidad2
ind_consultas_id
cant_personas_fam
cant_hijos_fam


In [2]:
datos

,id,profesion,actividad,est_civil,sexo_desc,tipo_nacionalidad,edad,estrato,ind_defuncion,ind_interd,...,ind_morosidad2,ind_consultas_id,status_salud_publica,tramo_salud_publico,cant_personas_fam,cant_vivos_fam,cant_hijos_fam,longitud,latitud,periodo
0,328458,,,0,0,CHILENO,26,5,V,0,...,0,0,0,X,3,3,1,-70.5813678.93E-4,-33.5633478.93E-4,2017
1,80319,,OBRERO,3,1,CHILENO,50,3,V,0,...,0,1,1,C,4,4,2,-72.3317648.93E-4,-38.2456978.93E-4,2017
2,378583,,,3,1,CHILENO,17,3,V,0,...,0,0,0,X,1,1,1,,,2017
3,76829,,EMPLEADO,3,1,CHILENO,53,1,V,0,...,0,0,1,C,5,5,3,-72.3237468.93E-4,-35.992268.93E-4,2017
4,9326,,,0,1,CHILENO,58,3,V,0,...,0,1,1,A,1,1,0,-70.6495138.93E-4,-33.493298.93E-4,2017
5,54194,,,0,1,CHILENO,23,3,V,0,...,0,0,1,A,3,3,1,,,2017
6,381330,,,3,1,CHILENO,16,3,V,0,...,0,0,0,X,1,1,1,,,2017
7,113087,,,0,0,CHILENO,26,3,V,0,...,0,1,1,A,3,3,1,,,2017
8,7736,,JUBILADO,3,0,CHILENO,96,3,V,0,...,0,0,1,B,2,2,1,-70.7532128.93E-4,-32.7364128.93E-4,2017
9,77009,,AUXILIAR DE PARVULOS,0,0,CHILENO,53,3,V,0,...,0,0,1,A,3,3,1,-71.2559668.93E-4,-33.5689898.93E-4,2017


In [28]:
#aplicar extraccion de caracteristicas por medio de valores sacados por
#parte la pagina
#http://www.emol.com/noticias/Economia/2016/04/02/796036/Como-se-clasifican-los-grupos-socioeconomicos-en-Chile.html?code=AQClmtYgqmiOyHwCBtU_QoNbcNdFU4HGlVUsctNH9rg1Rtke4yXnnSXZ9j3mp4FHGV3MjkAOUfajb495ByrlbuG6XIBkYDgSFZ4WeAwebtwSJe-eYyG8EPS-YHe7i-Y7_V2z7jzX6087YB38ENWq0jIxkRsp9WEhJaEakeZ_enCfkJQ_qa3kMP8f2Y6qNnJHDqdzaLAMsNzR3iipmRveLc9Z-fxspLb8OYeM7fn-uRSx_GYqQ95VgVIbXla5M_bqxsBB7eAu_X4gttX0LEPCsXtr12R3FmKbSTkpcqJWw60uAeh4A5N0yKORzoxLPglSkPuQ_RSEy0YoLF68_82v1opF#_=_
#abc1 = 4386000 c1a = 2070000 c1b = 1374000 c2 810000 c3 503000 d 307000 e 158000
#abc1 = 4386000*0.4038 + 2070000*0.2885 + 1374000*0.3076
#c2 = 810000 
#c3 = 503000 
#d = 307000 
#e = 158000
ab = 1415000
c1a = (808000+1414000)/2.0
c1b = (461000+460000)/2.0
abc1 = ab*0.4038 + c1a*0.2885 + c1b*0.3076
c2 = (259000+460000)/2.0
c3 = (135000+258000)/2.0
d = (670000+134000)/2.0
e = 66000


datos.loc[datos["estrato"] == 5 , "id"]= abc1*datos["cant_personas_fam"]
datos.loc[datos["estrato"] == 4 , "id"]= c2*datos["cant_personas_fam"]
datos.loc[datos["estrato"] == 3 , "id"]= c3*datos["cant_personas_fam"]
datos.loc[datos["estrato"] == 2 , "id"]= d*datos["cant_personas_fam"]
datos.loc[datos["estrato"] == 1 , "id"]= e*datos["cant_personas_fam"]

datos = datos.rename(columns={'id': 'ingreso_mensual'})


In [29]:
datos

,status_salud_publica,ingreso_mensual,edad,comuna,estrato,avaluo_auto,avaluo_bbrr,cant_personas_fam,cant_hijos_fam,tot_mont,ind_morosidad1,ind_morosidad2,n_rubros,ingreso_mensual
0,0,503000.0,55,0,3,0,0,4,3,0,0,0,0,786000.0
1,1,503000.0,67,13,3,0,0,3,2,0,0,0,1,589500.0
2,1,2790904.2,48,2,5,0,0,3,2,0,0,0,0,3100650.9
3,0,503000.0,39,25,3,0,0,3,0,0,0,0,1,589500.0
4,0,503000.0,88,1,3,0,0,2,0,0,0,0,1,393000.0
5,0,2790904.2,56,25,5,5180000,0,4,1,173,1,0,1,4134201.2
6,0,158000.0,29,10,1,0,0,1,0,1,1,0,0,66000.0
7,0,503000.0,90,1,3,0,0,1,0,0,0,0,0,196500.0
8,1,158000.0,72,14,1,0,0,5,2,0,0,0,0,330000.0
9,1,503000.0,33,4,3,0,2514944,4,2,0,0,0,0,786000.0


In [30]:
selectOpt = ['status_salud_publica', 'ingreso_mensual',
 'edad',
 'comuna',
 'estrato',
 'avaluo_auto',
 'avaluo_bbrr',
 'cant_personas_fam',
 'cant_hijos_fam',
 'tot_mont',
 'ind_morosidad1',
 'ind_morosidad2',
 'n_rubros']

datos = datos.loc[:,selectOpt]

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100,100, ), random_state=1, max_iter = 400)

datos = datos.loc[:,selectOpt]

#Simple K-Fold cross validation. 10 folds.
cv = cross_validation.KFold(len(datos), n_folds=5)

vp = 0
vn = 0
fp = 0
fn = 0
it = 0

#se inicia la validacion cruzada
for traincv, testcv in cv:
    
    probas = clf.fit(datos.loc[traincv].loc[:, selectOpt[1]:].values, datos.loc[traincv]['status_salud_publica'].values).predict(datos.loc[testcv].loc[:, selectOpt[1]:].values)
    print "iteracion: %i"%(it)
    for i in range(len(testcv)):
        if(datos.loc[testcv[i]]['status_salud_publica'] == 0 and probas[i] == datos.loc[testcv[i]]['status_salud_publica']):
            vp+=1
        elif(datos.loc[testcv[i]]['status_salud_publica'] == 0 and probas[i] != datos.loc[testcv[i]]['status_salud_publica']):
            fp+=1
        elif(datos.loc[testcv[i]]['status_salud_publica'] == 1 and probas[i] == datos.loc[testcv[i]]['status_salud_publica']):
            vn+=1
        elif(datos.loc[testcv[i]]['status_salud_publica'] == 1 and probas[i] != datos.loc[testcv[i]]['status_salud_publica']):
            fn+=1
    it+=1

#son mostrados los resultados
print "Exactitud: ",(vp+vn)/float(vp+vn+fp+fn)
print "%i | %i"%(vp,fp)
print "%i | %i"%(fn,vn)



iteracion: 0


KeyboardInterrupt: 

In [32]:
from sklearn.model_selection import ShuffleSplit


In [34]:
n_samples = datos.shape[0]
#cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
#cross_val_score(clf, iris.data, iris.target, cv=cv)

In [35]:
n_samples

887500

In [36]:
cv = ShuffleSplit(n_splits=1, test_size=0.3, random_state=0)

In [37]:
cv

ShuffleSplit(n_splits=1, random_state=0, test_size=0.3, train_size=None)

In [38]:
clf = MLPClassifier(solver='adam', activation = 'relu', alpha=1e-5, hidden_layer_sizes=(10, ), random_state=1)
#cv = ShuffleSplit(n_splits=2, test_size=0.3, random_state=0)
#scores = cross_val_score(clf, data.loc[:, "est_civil":], data['status_salud_publica'], cv=3)


In [42]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, datos.loc[:, "ingreso_mensual":], datos['status_salud_publica'], cv=3)

In [43]:
scores

array([ 0.56338014,  0.56338014,  0.56338057])